In [53]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import math
from sklearn.metrics  import confusion_matrix
from datetime import timedelta

In [54]:
filter_size1=5
num_filters1=16

filter_size2=5
num_filters2=36

fc_size=128

In [55]:
images_is = np.zeros((25000,784))
labels_is = np.zeros((25000, 10))
index = 0

In [56]:
for f in range(1,6):
	for w in range((50*(f-1))+1, ((50*(f-1))+1)+50):
		for i in range(1,11):
			for j in range(0,10):
				imgName = "MAHDBase_TrainingSet/Part{:02d}/writer{:03d}_pass{:02d}_digit{:d}".format(f,w,i,j)+".bmp"
				gray = cv2.imread(imgName,cv2.IMREAD_UNCHANGED)
				
				flattenedImage = gray.flatten() / 255.0
				images_is[index] = flattenedImage
				labels_is[index, j] = 1

				index+=1

In [57]:
data_train_images=images_is[0:18000,:]
data_vakidation_images=images_is[18001:20000,:]
data_test_images=images_is[20001:25000,:]

data_train_labels=labels_is[0:18000,:]
data_vakidation_labels=labels_is[18001:20000,:]
data_test_labels=labels_is[20001:25000]

In [58]:
data_test_cls=np.argmax(data_test_labels, axis=1)

In [59]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)
num_channels = 1
num_classes = 10

In [60]:
images = data_test_images[0:9]
cls_true = data_test_cls[0:9]

#plot_images function

In [61]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [62]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
    layer += biases
    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
    layer = tf.nn.relu(layer)
    return layer, weights

In [63]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

In [64]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer

In [65]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [66]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

layer_flat, num_features = flatten_layer(layer_conv2)

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

In [67]:
y_pred = tf.nn.softmax(layer_fc2)

y_pred_cls = tf.argmax(y_pred, dimension=1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [68]:
session = tf.Session()
session.run(tf.global_variables_initializer())

train_batch_size = 64
total_iterations = 0

BATCH_SIZE = 100

In [74]:
def optimize(num_iterations):
    global total_iterations
    start_time = time.time()

    #for i in range(total_iterations, total_iterations + num_iterations):
        
    for j in range(0, 20000, BATCH_SIZE):
            feed_dict_train = {x: data_train_images[j:j+BATCH_SIZE+1, :], y_true: data_train_labels[j:j+BATCH_SIZE+1, :]}
            session.run(optimizer, feed_dict=feed_dict_train)
        
            if i % 100 == 0:
                acc = session.run(accuracy, feed_dict=feed_dict_train)
                msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
                print(msg.format(i + 1, acc))
            
        #x_batch, y_true_batch = data.train.next_batch(train_batch_size)
        
        #feed_dict_train = {x: x_batch, y_true: y_true_batch}
        
        
            
    total_iterations += num_iterations
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [75]:
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,show_confusion_matrix=False):
    
    num_test = len(data_test_images)
    cls_pred = np.zeros(shape=num_test, dtype=np.int)
    i = 0
    while i < num_test:
        j = min(i + test_batch_size, num_test)
        images = data_test_images[i:j, :]
        labels = data_test_labels[i:j, :]
        feed_dict = {x: images, y_true: labels}
        
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        i = j
    cls_true = data_test_cls
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

In [73]:
print_test_accuracy()

Accuracy on Test-Set: 75.8% (3790 / 4999)


In [76]:
optimize(num_iterations=1)

Time usage: 0:01:15


In [78]:
optimize(num_iterations=99) # We already performed 1 iteration above.

Time usage: 0:01:13


In [79]:
print_test_accuracy()

Accuracy on Test-Set: 90.2% (4510 / 4999)


In [80]:
optimize(num_iterations=900) # We performed 100 iterations above.

Time usage: 0:01:15


In [81]:
print_test_accuracy()

Accuracy on Test-Set: 92.4% (4617 / 4999)
